In [40]:
%pip install gql

Note: you may need to restart the kernel to use updated packages.


In [41]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import pandas as pd

# Define the transport
transport = AIOHTTPTransport(url="http://localhost:8081/query")

# Create a GraphQL client
client = Client(transport=transport, fetch_schema_from_transport=True)

async def get_items() -> pd.DataFrame:
  # Define a query
  query = gql("""
  query getItems {
    items {
      edges {
          node {
              id
              name
              quantity
              category
              storageLocation {
                name
              }
          }
      }
    }
  }
  """)

  # Execute the query
  result = await client.execute_async(query)
  items = result['items']['edges']
  df = pd.DataFrame([item['node'] for item in items])
  df['storageLocation'] = df['storageLocation'].apply(lambda x: x['name'])
  return df

In [42]:
# issue a mutation to update the quantity of an item
async def update_quantity(id, quantity):
    mutation = gql("""
    mutation updateItem($id: ID!, $quantity: Int!) {
        updateItem(id: $id, input: {quantity: $quantity}) {
            id
            name
            quantity
        }
    }
    """)
    return await client.execute_async(mutation, variable_values={"id": id, "quantity": quantity})



In [50]:
from curses import raw


async def create_location(raw_name: str):
    split_str = raw_name.split('_')
    size_str = split_str[0]
    color_str = split_str[2]

    mutation = gql("""
    mutation createLocation($name: String!, $size: StorageLocationSize!, $color: StorageLocationColor!) {
        createStorageLocation(input: {name: $name, size: $size, color: $color}) {
            id
            name
        }
    }
    """)
    return await client.execute_async(mutation, variable_values={"name": raw_name, "size": size_str, "color": color_str})

In [51]:
df = await get_items()

In [52]:
await create_location('half_crate_orange_1')

{'createStorageLocation': {'id': '4294967301', 'name': 'half_crate_orange_1'}}